In [4]:
import cv2
import numpy as np
import tensorflow as tf

model = tf.keras.models.load_model("modelo_ganador.h5")
IMG_SIZE = 280

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("No se pudo abrir la cámara")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (IMG_SIZE, IMG_SIZE))
    img_norm = resized.astype("float32") / 255.0
    input_img = img_norm.reshape(1, IMG_SIZE, IMG_SIZE, 1)

    preds = model.predict(input_img, verbose=0)[0]
    winner = int(np.argmax(preds))

    display = frame.copy()

    #Proba 0–9
    y0, dy = 30, 25
    for i, p in enumerate(preds):
        text = f"{i}: {p*100:.1f}%"
        color = (0,255,0) if i == winner else (0,0,255)
        cv2.putText(display, text, (10, y0 + i*dy),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    cv2.putText(display, f"Pred: {winner}", (10, display.shape[0]-20),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)

    cv2.imshow("Original + probabilidades", display)

    cnn_vis = (img_norm*255).astype("uint8")
    cv2.imshow("Entrada a la CNN (280x280)", cnn_vis)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Modelo para leer diferentes numeros al mismo tiempo.

In [6]:
import cv2
import numpy as np
import tensorflow as tf

MODEL_PATH = "modelo_ganador.h5"  
model = tf.keras.models.load_model(MODEL_PATH)

IMG_SIZE = 280   

cap = cv2.VideoCapture(0) 

if not cap.isOpened():
    print("No se pudo abrir la cámara")
    exit()

print("OCR en tiempo real corriendo. Presiona 'q' para salir.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    display = frame.copy()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    #Blur 
    blur = cv2.GaussianBlur(gray, (5, 5), 0)

    #Binarización
    _, thresh = cv2.threshold(
        blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU
    )

    #Unir bordes
    kernel = np.ones((3, 3), np.uint8)
    dilated = cv2.dilate(thresh, kernel, iterations=2)

    #Quitar puntos
    clean = cv2.erode(dilated, kernel, iterations=1)

    #Contornos quita
    contours, hierarchy = cv2.findContours(
        clean, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )

    boxes = []
    for c in contours:
        x, y, w, h = cv2.boundingRect(c)

        area = w * h
        aspect_ratio = h / float(w)

        #Deimitar área
        if area < 600:
            continue

        #Min pixeles
        if w < 20 or h < 30:
            continue


        if aspect_ratio < 0.5 or aspect_ratio > 3.0:
            continue

        if w > frame.shape[1] * 0.7 or h > frame.shape[0] * 0.7:
            continue

        boxes.append((x, y, w, h))

    boxes = sorted(boxes, key=lambda b: b[0])

    for (x, y, w, h) in boxes:
        roi = clean[y:y+h, x:x+w]

        #Redimensiona
        roi_resized = cv2.resize(roi, (IMG_SIZE, IMG_SIZE))

        #Normaliza
        roi_norm = roi_resized.astype("float32") / 255.0
        roi_input = roi_norm.reshape(1, IMG_SIZE, IMG_SIZE, 1)

        #Predicción
        preds = model.predict(roi_input, verbose=0)[0]
        label = int(np.argmax(preds))
        prob  = float(np.max(preds))

        cv2.rectangle(display, (x, y), (x+w, y+h), (0, 255, 0), 2)

        text = f"{label} ({prob*100:.1f}%)"
        cv2.putText(display, text, (x, y - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6,
                    (0, 255, 0), 2, cv2.LINE_AA)

    #Tiempo real
    cv2.imshow("OCR en tiempo real - Digitos detectados", display)

    cv2.imshow("Umbral + dilatacion + erosion", clean)

    #Esc
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

OCR en tiempo real corriendo. Presiona 'q' para salir.
